In [2]:
import pandas as pd
from zipfile import Path as ZipPath
import matplotlib.pyplot as plt
from pathlib import Path
import pandas as pd
import numpy as np
from scipy.optimize import curve_fit
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

import matplotlib as mpl
mpl.use("pgf")

mpl.rcParams.update({
    "pgf.texsystem": "lualatex",
    "text.usetex": True,
    "font.family": "serif",
})
import matplotlib.pyplot as plt


# wczytywanie bezpośrednio z archiwum

In [3]:
fps = {x.stem: x for x in ZipPath('Dopasowanie.zip', at='/').iterdir()}
fps                                                                

{'IRL-2.2': Path('Dopasowanie.zip', 'IRL-2.2.csv'),
 'IRL-2.1': Path('Dopasowanie.zip', 'IRL-2.1.csv'),
 'trpp': Path('Dopasowanie.zip', 'trpp.csv')}

In [4]:
for k,v in fps.items():
    print(k, v.exists())

IRL-2.2 True
IRL-2.1 True
trpp True


In [5]:
%%time

dfs = {}
for k,v in fps.items():
    with v.open() as f:
        dfs[k] = pd.read_csv(f, parse_dates=['UTC'], index_col='UTC')


CPU times: user 1.22 s, sys: 38.8 ms, total: 1.26 s
Wall time: 1.27 s


# połączenie w jednego dataframe


In [6]:
df = pd.concat(
    [
        dfs["IRL-2.1"][["Flux, cps"]].rename(columns={"Flux, cps": "IRL-2.1"}),
        dfs["IRL-2.2"][["Flux, cps"]].rename(columns={"Flux, cps": "IRL-2.2"}),
        dfs["trpp"]
    ],
    axis=1
).sort_index()


In [7]:
print(df.columns.tolist())
print(dfs.keys())

['IRL-2.1', 'IRL-2.2', 'PLC_time', 'PAR', 'PK1', 'PK2', 'PK3', 'PK4', 'PK5', 'PK6', 'PB1', 'PB2', 'PB3', 'PB4', 'PB5', 'PB6']
dict_keys(['IRL-2.2', 'IRL-2.1', 'trpp'])


In [8]:

%%time
fig, (ax1, ax2) = plt.subplots(nrows=2, figsize=(12,12), sharex=True)

# GÓRA: IRL (tylko raz)
df.plot(
    ax=ax1,
    y=["IRL-2.1", "IRL-2.2"],
    logy=True,
    grid=True,
    marker=".",
)

# DÓŁ: TRPP
df.plot(
    ax=ax2,
    y=["PAR", "PK2", "PK6", "PB1"],
    grid=True,
    marker=".",
)

# ax1.legend()
# ax2.legend()
# plt.tight_layout()
# fig.savefig(f"{outdir}/całośc.png", format="png")  # bez tight

# ax1.legend()


CPU times: user 754 ms, sys: 27.9 ms, total: 782 ms
Wall time: 786 ms


<Axes: xlabel='UTC'>

In [9]:
intervals = {
    "zrzuty": slice("2025-12-08 19:55:30", "2025-12-08 21:30:30"),
    "zrzut01": slice("2025-12-08 19:55:30", "2025-12-08 20:03:30"),
    "zrzut02": slice("2025-12-08 20:07:30", "2025-12-08 20:11:30"),
    "zrzut03": slice("2025-12-08 20:15:30", "2025-12-08 20:19:30"),
    "zrzut04": slice("2025-12-08 20:23:30", "2025-12-08 20:27:30"),
    "zrzut05": slice("2025-12-08 20:31:30", "2025-12-08 20:35:30"),
    "zrzut06": slice("2025-12-08 20:39:30", "2025-12-08 20:43:30"),
    "zrzut07": slice("2025-12-08 20:47:30", "2025-12-08 20:51:30"),
    "zrzut08": slice("2025-12-08 20:55:30", "2025-12-08 20:59:30"),
    "zrzut09": slice("2025-12-08 21:00:30", "2025-12-08 21:07:30"),
    "zrzut10": slice("2025-12-08 21:08:30", "2025-12-08 21:12:30"),
    "zrzut11": slice("2025-12-08 21:12:30", "2025-12-08 21:18:30"),
}

[(k, v.start, v.stop) for k, v in intervals.items()]


[('zrzuty', '2025-12-08 19:55:30', '2025-12-08 21:30:30'),
 ('zrzut01', '2025-12-08 19:55:30', '2025-12-08 20:03:30'),
 ('zrzut02', '2025-12-08 20:07:30', '2025-12-08 20:11:30'),
 ('zrzut03', '2025-12-08 20:15:30', '2025-12-08 20:19:30'),
 ('zrzut04', '2025-12-08 20:23:30', '2025-12-08 20:27:30'),
 ('zrzut05', '2025-12-08 20:31:30', '2025-12-08 20:35:30'),
 ('zrzut06', '2025-12-08 20:39:30', '2025-12-08 20:43:30'),
 ('zrzut07', '2025-12-08 20:47:30', '2025-12-08 20:51:30'),
 ('zrzut08', '2025-12-08 20:55:30', '2025-12-08 20:59:30'),
 ('zrzut09', '2025-12-08 21:00:30', '2025-12-08 21:07:30'),
 ('zrzut10', '2025-12-08 21:08:30', '2025-12-08 21:12:30'),
 ('zrzut11', '2025-12-08 21:12:30', '2025-12-08 21:18:30')]

In [10]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

for name, sl in intervals.items():

    df_cut = df.loc[sl]
    if name == "zrzuty":
        continue
    if df_cut.empty:
        print(f"{name}: pusty zakres")
        continue

    fig, (ax1, ax2) = plt.subplots(
        nrows=2,
        figsize=(16, 12),
        sharex=True
    )

    # ===== GÓRA: IRL =====
    df_cut.plot(
        ax=ax1,
        y=["IRL-2.1"],
        logy=True,
        grid=True,
        marker=".",
    )

    # ===== DÓŁ: PK6 =====
    df_cut.plot(
        ax=ax2,
        y=["PK6"],
        grid=True,
        marker=".",
    )

    # format osi X
    ax1.set_title(f"{name} | {sl.start} → {sl.stop}")
    ax2.set_xlabel("UTC")

    plt.tight_layout()
    plt.show()



/tmp/ipykernel_35356/102232361.py:41: UserWarning: FigureCanvasPgf is non-interactive, and thus cannot be shown
  plt.show()
/tmp/ipykernel_35356/102232361.py:41: UserWarning: FigureCanvasPgf is non-interactive, and thus cannot be shown
  plt.show()
/tmp/ipykernel_35356/102232361.py:41: UserWarning: FigureCanvasPgf is non-interactive, and thus cannot be shown
  plt.show()
/tmp/ipykernel_35356/102232361.py:41: UserWarning: FigureCanvasPgf is non-interactive, and thus cannot be shown
  plt.show()
/tmp/ipykernel_35356/102232361.py:41: UserWarning: FigureCanvasPgf is non-interactive, and thus cannot be shown
  plt.show()
/tmp/ipykernel_35356/102232361.py:41: UserWarning: FigureCanvasPgf is non-interactive, and thus cannot be shown
  plt.show()
/tmp/ipykernel_35356/102232361.py:41: UserWarning: FigureCanvasPgf is non-interactive, and thus cannot be shown
  plt.show()
/tmp/ipykernel_35356/102232361.py:41: UserWarning: FigureCanvasPgf is non-interactive, and thus cannot be shown
  plt.show()


In [11]:
df_cut = df.loc[intervals["zrzut01"]]

fig, (ax1, ax2) = plt.subplots(nrows=2, figsize=(12,12), sharex=True)

df_cut.plot(
    ax=ax1,
    y=["IRL-2.1"],
    logy=True,
    grid=True,
    marker=".",
)

df_cut.plot(
    ax=ax2,
    y=["PK6"],
    grid=True,
    marker=".",
)

plt.tight_layout()
plt.savefig 
plt.show()


/tmp/ipykernel_35356/3702980905.py:22: UserWarning: FigureCanvasPgf is non-interactive, and thus cannot be shown
  plt.show()


In [12]:
df_cut = df.loc[intervals["zrzut01"]]

fig, (ax1, ax2) = plt.subplots(nrows=2, figsize=(12,12), sharex=True)

df_cut.plot(
    ax=ax1,
    y=["IRL-2.2"],
    logy=True,
    grid=True,
    marker=".",
)

df_cut.plot(
    ax=ax2,
    y=["PK6"],
    grid=True,
    marker=".",
)

plt.tight_layout()
plt.show()


/tmp/ipykernel_35356/4119963235.py:21: UserWarning: FigureCanvasPgf is non-interactive, and thus cannot be shown
  plt.show()


In [13]:
cuts = {
    "zrzut01": ("2025-12-08 19:57:30", "2025-12-08 20:01:30"),
   
}

for name, (t0, t1) in cuts.items():
    maly_df = df.loc[t0:t1]

    t = (maly_df.index - maly_df.index[0]).total_seconds()
    y = maly_df["IRL-2.2"]
    

In [14]:
cuts = {
    "zrzut01": ("2025-12-08 19:57:30", "2025-12-08 20:01:30"),
    "zrzut02": ("2025-12-08 20:08:40", "2025-12-08 20:09:45"),
    "zrzut03": ("2025-12-08 20:17:15", "2025-12-08 20:18:05"),
    "zrzut04": ("2025-12-08 20:25:05", "2025-12-08 20:26:10"),
    "zrzut05": ("2025-12-08 20:32:48", "2025-12-08 20:33:43"),
    "zrzut06": ("2025-12-08 20:41:08", "2025-12-08 20:41:53"),
    "zrzut07": ("2025-12-08 20:49:13", "2025-12-08 20:50:08"),
    "zrzut08": ("2025-12-08 20:56:33", "2025-12-08 20:57:33"),
    "zrzut09": ("2025-12-08 21:02:50", "2025-12-08 21:03:40"),
    "zrzut10": ("2025-12-08 21:09:22", "2025-12-08 21:10:12"),
    "zrzut11": ("2025-12-08 21:14:59", "2025-12-08 21:15:34"),
}

for name, (t0, t1) in cuts.items():
    maly_df = df.loc[t0:t1]

    t = (maly_df.index - maly_df.index[0]).total_seconds()
    y = maly_df["IRL-2.2"]

In [15]:
t

Index([         0.0,  0.023238897,  0.099303007,  0.119539976,  0.199012995,
        0.220527888,  0.299893856,  0.320500851,  0.400672913,  0.423434973,
       ...
       35.500520945, 35.521129847, 35.600239039,  35.62114501,  35.70023799,
       35.720597983, 35.799998045, 35.820545912, 35.899873972, 35.922618866],
      dtype='float64', name='UTC', length=754)

In [16]:

# def lin_fit(t,x1,x2,a,b,c,T,x):   # funkcja do segmentu liniowego
#     y = a*t+b
#     return y

# def par_fit(t,x1,x2,a,b,c,T,x):    # funkcja do segmentu paraboli
#     d=-2*c*x1
#     f = c*x2**2 + d*x2
#     y = c*t**2 *d*t + f
#     return y

# # def exp_fit(t,x1,x2,a,b,c,T,x):   # funkcja do segmentu exp
# #     d=-2*c*x1
# #     f = c*x2**2 + d*x2
# #     g = (c*x2**2 + d*x2+f)/np.exp(x2/T)
# #     y = np.exp(t/T)*g
# #     return y

# def exp_fit(t, a, T):   # funkcja do segmentu exp
#     y = np.exp(t/T) *a
#     return y



In [17]:


def exp_fit(t, a, T):
    return a * np.exp(t / T)

rows = []

for zrzut, (t0, t1) in cuts.items():
    wycinek = df.loc[t0:t1, ["IRL-2.1", "IRL-2.2"]]

    for ch in ["IRL-2.1", "IRL-2.2"]:
        s = wycinek[ch].dropna()
        if s.empty:
            rows.append([zrzut, ch, np.nan, np.nan])
            continue

        t = (s.index - s.index[0]).total_seconds().to_numpy()
        y = s.to_numpy()

        # opcjonalnie: odetnij niefizyczne wartości (np. <=0 przy logu)
        m = np.isfinite(t) & np.isfinite(y)
        t, y = t[m], y[m]
        if len(y) < 3:
            rows.append([zrzut, ch, np.nan, np.nan])
            continue

        # FIT
        popt, pcov = curve_fit(
            exp_fit, t, y,
            p0=[y[0], 60.0],
            maxfev=20000
        )

        a_hat = popt[0]
        sigma_a = float(np.sqrt(pcov[0, 0]))

        rows.append([zrzut, ch, a_hat, sigma_a])

res_long = pd.DataFrame(rows, columns=["zrzut", "kanal", "a_hat", "sigma_a"])

# szeroko: osobne kolumny dla IRL-2.1 i IRL-2.2
res_wide = (
    res_long
    .pivot(index="zrzut", columns="kanal", values=["a_hat", "sigma_a"])
    .sort_index()
)

# spłaszcz nazwy kolumn: ('a_hat','IRL-2.1') -> 'a_hat_IRL-2.1'
res_wide.columns = [f"{a}_{ch}" for a, ch in res_wide.columns]
res_wide = res_wide.reset_index()

print(res_wide[["zrzut",
                "a_hat_IRL-2.1","sigma_a_IRL-2.1",
                "a_hat_IRL-2.2","sigma_a_IRL-2.2"]])




      zrzut  a_hat_IRL-2.1  sigma_a_IRL-2.1  a_hat_IRL-2.2  sigma_a_IRL-2.2
0   zrzut01   3.604681e+06       677.516640   17462.680839         5.406253
1   zrzut02   3.707318e+06      3141.407583   18363.023001        17.217121
2   zrzut03   3.929972e+06      2964.725064   20104.093871        19.273687
3   zrzut04   3.329523e+06      3452.919374   17443.215798        18.662614
4   zrzut05   3.390601e+06      4328.790212   18381.517871        26.450618
5   zrzut06   3.583915e+06      3071.023020   19893.074954        19.692824
6   zrzut07   3.402673e+06      3044.787130   19524.260767        21.377803
7   zrzut08   3.342748e+06      2595.585414   19653.207093        17.222456
8   zrzut09   3.610857e+06      2541.099569   21756.557232        18.327930
9   zrzut10   3.306605e+06      3479.440228   20469.124964        24.041782
10  zrzut11   4.146025e+06      2027.550605   26383.687785        20.794764


Funkcja: scipy.optimize.curve_fit – nieliniowe MNK (Levenberg–Marquardt domyślnie).
exp_fit – Twoja funkcja modelu 
f(t; a,T) zwracająca wartościdopasowania dla wektora t
t – wektor/seria argumentu niezależnego (np. czas).
y – wektor danych obserwowanych (zmienna zależna).
p0=[y.iloc[0], 60] – zgad startowy parametrów [alfa i T ]
maxfev=10000 – maks. liczba wywołań funkcji modelu (zwiększa szansę zbieżności).

In [18]:
popt, pcov = curve_fit(exp_fit, t,y, p0=[y[0], 60], maxfev=10000)
popt, np.sqrt(np.diag(pcov)) # pierwszy to parametry a i T drugi to ich niepewności

(array([26383.68778465,    67.91012883]), array([20.79476392,  0.15553792]))

popt – wektor estymowanych parametrów
pcov – przybliżona macierz kowariancji estymatorów (Podaje nam wartość z moich danych odchylenia standardowego, ale zwraca jest jako simga^2 więc trzeba ją wstaić pod pierwiastek drugiego stopnia )

In [19]:
sigma_a, sigma_T = np.sqrt(np.diag(pcov))   # Δa, ΔT
a_hat, T_hat = popt                         # a, T

In [20]:
print(f"a = {a_hat:.6g} ± {sigma_a:.2g}")
print(f"T = {T_hat:.6g} ± {sigma_T:.2g} [h]")


a = 26383.7 ± 21
T = 67.9101 ± 0.16 [h]


In [21]:
fig, ax = plt.subplots(figsize=(20,12))
ax.plot(t, y, marker='.', label='data')
ax.plot(t, exp_fit(t, *popt), 'r-', label='fit')
ax.set_yscale('log')
ax.grid(True, which='both', alpha=0.3)
ax.legend()

In [22]:
# plt.figure(figsize=(20,12))
# plt.plot(t, y, marker='.', label=cn)
# plt.plot(t, exp_fit(t, *popt), 'r-', label='fit')
# plt.axvline(x= popt[0], color='gray', linestyle='--')
# plt.axvline(x= popt[1], color='gray', linestyle=':')
# plt.grid(True, alpha=0.3)
# plt.legend()
# plt.xlabel('Czas, s')
# plt.ylabel('Liczba zliczen, cps')
# plt.tight_layout()
# plt.show()
# # (opcjonalnie) szybka diagnostyka dopasowania
# res = y - exp_fit(t, *popt)
# print("Parametry (x1, x2_raw, a, b, p, c, T) =", [f"{v:.3g}" for v in popt])
# print(f"RMSE = {np.sqrt(np.mean(res**2)):.6g}, MAE = {np.mean(np.abs(res)):.6g}")


Numer telefonu do tomka